## Get API call intergration in angular

Angular has it's own HttpClient library. If you want to use it, first, you have to add it to the providers in app.config.ts file.

In [ ]:
# app.config.ts

import { ApplicationConfig, provideBrowserGlobalErrorListeners, provideZoneChangeDetection } from '@angular/core';
import { provideRouter } from '@angular/router';

import { routes } from './app.routes';
import { provideHttpClient } from '@angular/common/http';

export const appConfig: ApplicationConfig = {
  providers: [
    provideBrowserGlobalErrorListeners(),
    provideRouter(routes),
    provideHttpClient(),  ## <--- add this code (in Angular 21 it's not required ), 
                          ## but you can add this if you want additional functionality
  ]
};

Then, we have to create an instance of client service by using "injection" in component file.

In [ ]:
import { HttpClient } from '@angular/common/http';
import { Component, inject } from '@angular/core';

@Component({
  selector: 'app-get-api',
  imports: [],
  templateUrl: './get-api.html',
  styleUrl: './get-api.css',
})
export class GetApi {

  http = inject(HttpClient);  ## <--- client service injection
    
}

Now, we can create a method to "fetch" users from API.

In [ ]:
import { HttpClient } from '@angular/common/http';
import { Component, inject, OnInit } from '@angular/core';

@Component({
  selector: 'app-get-api',
  imports: [],
  templateUrl: './get-api.html',
  styleUrl: './get-api.css',
})
export class GetApi implements OnInit {  ## <--- implement OnInit to trigger method after component initialization

  http = inject(HttpClient);
  userList: any[] = [];  ## <--- I used signal for storing the fetched data

  ngOnInit(): void {  ## <--- invoke method after initialization
    this.getUsers();
  }

  getUsers() {
    this.http.get("https://api.freeprojectapi.com/api/GoalTracker/getAllUsers").subscribe((res: any) => {
      this.usersList.set(res);
    });
  }
    
  ## better way to fetch data and control responce
  
  getUsers() {
    this.http.post("https://api.freeprojectapi.com/api/GoalTracker/getAllUsers").subscribe({
      next: (result:any) => {
        this.usersList = res;
        console.log("users succesfully fetched")
      },
      error: (error: any) => {
        console.log("couldn't fetch data", error)
      }
    })
  }

}

## html

<div class="container-fluid mt-4">
  <div class="row g-4">

    <!-- TABLE COLUMN -->
    <div class="col-md-8">
      <div class="card shadow-sm">
        <div class="card-header bg-warning fw-bold">
          Batch List
        </div>
        <div class="card-body">
          <table class="table table-bordered table-hover align-middle">
            <thead class="table-light">
              <tr>
                <th>Batch ID</th>
                <th>Batch Name</th>
                <th>Created Date</th>
                <th style="width: 120px;">Actions</th>
              </tr>
            </thead>
            <tbody>
              @for (item of allBatches(); track $index) {
                  <tr>
                    <td>{{$index + 1}}</td>
                    <td>{{item.batchName}}</td>
                    <td>{{item.createdDate}}</td>
                    <td>
                      <button class="btn btn-sm btn-primary" (click)="onEditBatch(item)">Edit</button>
                      <button class="btn btn-sm btn-danger" (click)="onDeleteBatch(item.batchId)">Delete</button>
                    </td>
                  </tr>
              }
              <!-- rows dynamically added -->
            </tbody>
          </table>
        </div>
      </div>
    </div>
    <p>allBatches length = {{allBatches().length}}</p>

    <!-- FORM COLUMN -->
    <div class="col-md-5">
      <div class="card shadow-sm">
        <div class="card-header bg-warning fw-bold">
          Create / Update Batch
        </div>
        <div class="card-body">
          <form>

            <div class="mb-3">
              <label for="batchName" class="form-label">Batch Name</label>
              <input name="batchName" type="text" [(ngModel)]="newBatchObj.batchName" class="form-control" placeholder="Enter batch name">
            </div>

            <div class="mb-3">
              <label for="createdDate" class="form-label">Created Date</label>
              <input name="createdDate" type="datetime-local" [(ngModel)]="newBatchObj.createdDate" class="form-control">
            </div>

            <div class="d-flex gap-2">
              @if (newBatchObj.batchId === 0) {
                <button type="submit" (click)="onSaveBatch()" class="btn btn-success">Save</button>
              } @else if (newBatchObj.batchId != 0) {
                <button type="submit" (click)="onUpdateBatch()" class="btn btn-success">Edit</button>
              }
              <button type="reset" class="btn btn-secondary">Clear</button>
            </div>
          </form>
        </div>
      </div>
    </div>

  </div>
</div>

Above method is not working as I wanted to, because the fetched data didn't display even when I used ngOnInit(). To avoid such unpleasant situation I used signal.

In [ ]:
import { HttpClient } from '@angular/common/http';
import { Component, inject, OnInit, signal } from '@angular/core';
import { FormsModule } from '@angular/forms';

@Component({
  selector: 'app-batch-master',
  imports: [FormsModule, ],
  templateUrl: './batch-master.html',
  styleUrl: './batch-master.css',
})
export class BatchMaster {

  ngOnInit(): void {
    this.getAllBatches();
  }
  
  allBatches = signal<Batch[]>([]); ## <-- signal variable
  http = inject(HttpClient)
  
  getAllBatches() {
    this.http.get("https://api.freeprojectapi.com/api/FeesTracking/batches").subscribe({
      next: (res: any) => {
        this.allBatches.set(res);
        console.log(this.allBatches)
      },
      error: (error: any) => {
        console.log("something went wrong", error.error.message)
      }
    })
  }

}

class Batch {
  batchId: number;
  batchName: string;
  createdDate: Date;

  constructor() {
    this.batchId = 0;
    this.batchName = "",
    this.createdDate = new Date();
  }
}

### POST api

For this part of the chapter I will use FreeProjectApi.com -> FeesTracking. 

In [ ]:
{
  "batchId": 0,
  "batchName": "string",
  "createdDate": "2026-01-31T06:28:49.213Z"
}

To post the object we need first to create it. We could use standard code like below

In [ ]:
## this is an example, don't do like this!!!!!!!

newBatchObj: any = {
    batchId: 0,
    batchName: "sampleName",
    createdDate: new Date()
}

But the proper way to do this is by creating a class of the object we want to add. Here is an example:

In [ ]:
import { Component } from '@angular/core';

@Component({
  selector: 'app-batch-master',
  imports: [FormsModule],  ## <-- import FormsModule
  templateUrl: './batch-master.html',
  styleUrl: './batch-master.css',
})
export class BatchMaster {

  newBatchObj: Batch = new Batch(); ## <-- new instance of created class of the object

}

## create a class Batch (it can be in a different folder, doesn't have to be in
## the component)
class Batch {
  ## the same properties as the object we want to POST
  batchId: number;
  batchName: string;
  createdDate: Date;

  ## constructor to initialize the class
  constructor() {
    this.batchId = 0;
    this.batchName = "",
    this.createdDate = new Date();
  }
}

Now, create a method to save the object

In [ ]:
onSaveBatch() {
    ## two arguments, endpoint address and the object we want to pass
    this.http.post("https://api.freeprojectapi.com/api/FeesTracking/batches", this.newBatchObj).subscribe({
      next: (result:any) => {
        this.getAllBatches(); ##  refresh page and results on page
      },
      error: (error: any) => {
        console.log(error, "error here")
      }
    })
    }

### PUT method

Let's say we have a fetched data with few items and we want to edit one of them. Each item has an edit and delete button.



In [ ]:
<tbody>
  @for (item of allBatches(); track $index) {
      <tr>
        <td>{{$index + 1}}</td>
        <td>{{item.batchName}}</td>
        <td>{{item.createdDate}}</td>
        <td>
          
          <button class="btn btn-sm btn-primary" (click)="onEditBatch(item)">Edit</button>
        </td>
      </tr>
  }
</tbody>

Now, we can create a method which will replace the empty input field with the data we want to edit.

In [ ]:
onEditBatch (data: Batch) {
    # if we use this.newBatchObj = data instead below code, we will have
    # displayed value and when we change it in input field, the displayed value
    # will also change. To prevent this we have "disconnect" input from display value.
    const stringData = JSON.stringify(data); 
    const strObj = JSON.parse(stringData);
    this.newBatchObj = strObj;
}

Now, we can change the input value and add a method which will replace old data with a new one

In [ ]:
onUpdateBatch() {
    ## we pass endpoint with id and the new object we want to pass
    this.http.put(`https://api.freeprojectapi.com/api/FeesTracking/batches/${this.newBatchObj.batchId}`, this.newBatchObj).subscribe({
      next: (res: any) => {
        console.log("succesfully updated")
        this.getAllBatches(); ## refresh page
        this.newBatchObj = new Batch(); ## reset form
      }
    })
  }